# Explore here

In [288]:
# Your code here
import pandas as pd

url = ("https://raw.githubusercontent.com/4GeeksAcademy/NLP-project-tutorial/main/url_spam.csv")
df = pd.read_csv(url)

df.head()

,url,is_spam
0,https://briefingday.us8.list-manage.com/unsubs...,True
1,https://www.hvper.com/,True
2,https://briefingday.com/m/v4n3i4f3,True
3,https://briefingday.com/n/20200618/m#commentform,False
4,https://briefingday.com/fan,True


In [289]:
print(df.dtypes)

url        object
is_spam      bool
dtype: object


In [290]:
# Convert labels to binary
df['is_spam'] = df['is_spam'].map({True: 0, False: 1})
df.head()

,url,is_spam
0,https://briefingday.us8.list-manage.com/unsubs...,0
1,https://www.hvper.com/,0
2,https://briefingday.com/m/v4n3i4f3,0
3,https://briefingday.com/n/20200618/m#commentform,1
4,https://briefingday.com/fan,0


In [291]:
print(df.shape)
print(f"True: {len(df.loc[df.is_spam == 1])}")
print(f"False: {len(df.loc[df.is_spam == 0])}")

(2999, 2)
True: 2303
False: 696


In [292]:
df = df.drop_duplicates()
df = df.reset_index(inplace = False, drop = True)
df.shape

(2369, 2)

Approx 600 repeated records eliminated

In [293]:
pip install regex

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.0.1 -> 24.1.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [294]:
import regex as re

def preprocess_text(text):
    # Remove any character that is not a letter (a-z) or white space ( )
    text = re.sub(r'[^a-z ]', " ", text)
    
    # Remove white spaces
    text = re.sub(r'\s+[a-zA-Z]\s+', " ", text)
    text = re.sub(r'\^[a-zA-Z]\s+', " ", text)

    # Multiple white spaces into one
    text = re.sub(r'\s+', " ", text.lower())

    # Remove tags
    text = re.sub("&lt;/?.*?&gt;"," &lt;&gt; ", text)

    return text

df["url"] = df["url"].apply(preprocess_text)
df.head()

,url,is_spam
0,https briefingday us list manage com unsubscribe,0
1,https www hvper com,0
2,https briefingday com v i,0
3,https briefingday com m commentform,1
4,https briefingday com fan,0


In [295]:
pip install nltk

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.0.1 -> 24.1.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [296]:
from nltk import download
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
download("wordnet")
download("stopwords")

lemmatizer = WordNetLemmatizer()
stop_words = stopwords.words("english")

def lemmatize_text(message, lemmatizer = lemmatizer):
    words = message.split()
    tokens = [lemmatizer.lemmatize(word) for word in words]
    tokens = [word for word in tokens if word not in stop_words]
    return " ".join(tokens)

df["url"] = df["url"].apply(lemmatize_text)
df.head()

[nltk_data] Downloading package wordnet to /home/vscode/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /home/vscode/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,url,is_spam
0,http briefingday u list manage com unsubscribe,0
1,http www hvper com,0
2,http briefingday com v,0
3,http briefingday com commentform,1
4,http briefingday com fan,0


In [297]:
#train/test split
from sklearn.model_selection import train_test_split

X = df['url']
y = df['is_spam']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [298]:
#Feature Extraction

from sklearn.feature_extraction.text import TfidfVectorizer

# Initialize the TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer(max_features=5000, max_df=0.8, min_df=5)

# Fit the TF-IDF vectorizer on the training data
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)

# Transform the test data using the fitted vectorizer
X_test_tfidf = tfidf_vectorizer.transform(X_test)

In [299]:
# Get the feature names
feature_names = tfidf_vectorizer.get_feature_names_out()

# Create a DataFrame
pd.DataFrame(X_train_tfidf.toarray(), columns=feature_names).head()

,aa,abcnews,abortion,ac,account,action,ad,administration,advertising,ae,...,world,wsj,wwdc,www,year,york,young,youtu,youtube,zone
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.108581,0.000000,0.42103,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.140960,0.000000,0.00000,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.000000,0.000000,0.00000,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.000000,0.614245,0.00000,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.366757,0.0,0.0,0.000000,0.000000,0.00000,0.0,0.0,0.0,0.0


In [300]:
#Model Building
from sklearn.svm import SVC

# Initialize the SVM classifier
model = SVC(kernel='linear')

# Train the model
model.fit(X_train_tfidf, y_train)

SVC(kernel='linear')

In [301]:
#Model Evaluation
# Predict on the test set
y_pred = model.predict(X_test_tfidf)
y_pred

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1,
       1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

In [302]:
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

# Print accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.4f}')

Accuracy: 0.9304


In [303]:
import numpy as np
from sklearn.model_selection import GridSearchCV

hyperparams = {
    "C": [0.001, 0.01, 0.1, 1, 10, 100, 1000],
    "kernel": ["linear", "poly", "rbf", "sigmoid"],
    "degree": [1, 2, 3, 4, 5],
    "gamma": ["scale", "auto"]
}

# We initialize the random search
grid = GridSearchCV(model, hyperparams, scoring = "accuracy", cv = 5)
grid

GridSearchCV(cv=5, estimator=SVC(kernel='linear'),
             param_grid={'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000],
                         'degree': [1, 2, 3, 4, 5], 'gamma': ['scale', 'auto'],
                         'kernel': ['linear', 'poly', 'rbf', 'sigmoid']},
             scoring='accuracy')

In [304]:
grid.fit(X_train_tfidf, y_train)

print(f"Best hyperparameters: {grid.best_params_}")

Best hyperparameters: {'C': 1, 'degree': 2, 'gamma': 'scale', 'kernel': 'poly'}


In [305]:
opt_model = SVC(C = 1, degree = 2, gamma = "scale", kernel = "poly", random_state = 42)
opt_model.fit(X_train_tfidf, y_train)
y_pred = opt_model.predict(X_test_tfidf)
accuracy_score(y_test, y_pred)

0.930379746835443

In [306]:
from pickle import dump

dump(model, open("../models/svm_classifier_C-1_deg-2_gam-scale_ker-poly_42.sav", "wb"))